In [ ]:
# !pip install sentencepiece
#!pip install polars
# !pip install transformers[torch]
# !pip install "accelerate>=0.26.0"


In [2]:
import polars as pl
import torch
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          Seq2SeqTrainer,
                          Seq2SeqTrainingArguments,
                          DataCollatorForSeq2Seq)
from datasets import load_dataset, Dataset, DatasetDict
import sentencepiece
import accelerate
from huggingface_hub import login

In [ ]:
# Load environment variables (API keys) from .env file
load_dotenv()
HUGGINFACE_TOKEN = os.getenv("HUGGINFACE_TOKEN")

In [3]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("BramVanroy/ul2-small-dutch-simplification-mai-2023", legacy=False)
model = AutoModelForSeq2SeqLM.from_pretrained("BramVanroy/ul2-small-dutch-simplification-mai-2023")

C:\Users\ElinedeKinkelder\anaconda3\Lib\site-packages\transformers\convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
df = pl.read_parquet('hf://datasets/UWV/Leesplank_NL_wikipedia_simplifications_preprocessed/data/train-*.parquet')
print(df.columns)
ds = Dataset.from_pandas(df.to_pandas())

In [ ]:
# Check if you have a GPU, otherwise default to CPU
if torch.backends.mps.is_available():  # Check for AMD ROCm GPU
    device = torch.device("mps")
    print("mps")
else:
    device = torch.device("cpu")
    print("cpu")
model.to(device)


def tokenize_function(examples):
    return tokenizer(examples["prompt"], text_target=examples["result"], padding="max_length", truncation=True)

# Tokenize datasets
#tokenized_datasets = ds.map(tokenize_function, batched=True)
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=['prompt', 'result', '__index_level_0__'])

In [ ]:
# Split the tokenized dataset (e.g., 80% train, 20% validation)
splits = tokenized_datasets.train_test_split(test_size=0.2)

# Check the split datasets
train_dataset = splits["train"]
valid_dataset = splits["test"]
 


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Enable mixed precision training on ROCm-enabled AMD GPUs
    save_strategy="epoch"
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

In [ ]:
# Save the model
login('HUGGINGFACE_TOKEN')
trainer.push_to_hub("UWV/ul2-small-dutch-simplification-okt-2024")